In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e12/sample_submission.csv
/kaggle/input/playground-series-s4e12/train.csv
/kaggle/input/playground-series-s4e12/test.csv


In [2]:
!python -m pip install --upgrade pip
!python -m pip install autogluon

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0
  Preparing metadata (setup.py) ... - \ | done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 41.7 

# Metric

In [3]:
from sklearn.metrics import root_mean_squared_log_error
from autogluon.core.metrics import make_scorer

root_mean_squared_log_scorer = make_scorer(name='root_mean_squared_log_error',
                                 score_func=root_mean_squared_log_error,
                                 optimum=0,
                                 greater_is_better=False)

# Training

In [4]:
from autogluon.tabular import TabularDataset, TabularPredictor

train_data = TabularDataset('/kaggle/input/playground-series-s4e12/train.csv')
train_data.head()

Loaded data from: /kaggle/input/playground-series-s4e12/train.csv | Columns = 21 / 21 | Rows = 1200000 -> 1200000


,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,...,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,...,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0


In [5]:
label = 'Premium Amount'
train_data[label].describe()

count    1.200000e+06
mean     1.102545e+03
std      8.649989e+02
min      2.000000e+01
25%      5.140000e+02
50%      8.720000e+02
75%      1.509000e+03
max      4.999000e+03
Name: Premium Amount, dtype: float64

In [6]:
time_limit = 28800
metric = root_mean_squared_log_scorer 
predictor = TabularPredictor(label, eval_metric=metric).fit(train_data, time_limit=time_limit, presets='best_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20241214_170447"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.14
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       29.77 GB / 31.35 GB (95.0%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fi

(_ray_fit pid=613) [1000]	valid_set's l2: 708913	valid_set's root_mean_squared_log_error: -1.13947
(_ray_fit pid=613) [2000]	valid_set's l2: 708392	valid_set's root_mean_squared_log_error: -1.13869 [repeated 4x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(_ray_fit pid=611) [3000]	valid_set's l2: 698743	valid_set's root_mean_squared_log_error: -1.13462 [repeated 4x across cluster]
(_ray_fit pid=613) [4000]	valid_set's l2: 708396	valid_set's root_mean_squared_log_error: -1.13797 [repeated 4x across cluster]
(_ray_fit pid=613) [5000]	valid_set's l2: 708564	valid_set's root_mean_squared_log_error: -1.13773 [repeated 4x across cluster]


(_dystack pid=231) 	Warning: Exception caused LightGBMXT_BAG_L1 to fail during training... Skipping this model.
(_dystack pid=231) 		ray::_ray_fit() (pid=612, ip=172.19.2.2)
(_dystack pid=231)   File "/opt/conda/lib/python3.10/site-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 413, in _ray_fit
(_dystack pid=231)     fold_model.fit(X=X_fold, y=y_fold, X_val=X_val_fold, y_val=y_val_fold, time_limit=time_limit_fold, **resources, **kwargs_fold)
(_dystack pid=231)   File "/opt/conda/lib/python3.10/site-packages/autogluon/core/models/abstract/abstract_model.py", line 925, in fit
(_dystack pid=231)     out = self._fit(**kwargs)
(_dystack pid=231)   File "/opt/conda/lib/python3.10/site-packages/autogluon/tabular/models/lgb/lgb_model.py", line 283, in _fit
(_dystack pid=231)     self.model = train_lgb_model(early_stopping_callback_kwargs=early_stopping_callback_kwargs, **train_params)
(_dystack pid=231)   File "/opt/conda/lib/python3.10/site-packages/autogluon/tabular/

(_ray_fit pid=741) [1000]	valid_set's l2: 694714	valid_set's root_mean_squared_log_error: -1.13081 [repeated 4x across cluster]
(_ray_fit pid=1054) [1000]	valid_set's l2: 696774	valid_set's root_mean_squared_log_error: -1.12803


(_dystack pid=231) 	-1.1311	 = Validation score   (-root_mean_squared_log_error)
(_dystack pid=231) 	208.36s	 = Training   runtime
(_dystack pid=231) 	42.57s	 = Validation runtime
(_dystack pid=231) Fitting model: RandomForestMSE_BAG_L1 ... Training model for up to 3962.66s of the 6358.94s of remaining time.
(_dystack pid=231) 	-1.1284	 = Validation score   (-root_mean_squared_log_error)
(_dystack pid=231) 	1574.53s	 = Training   runtime
(_dystack pid=231) 	37.09s	 = Validation runtime
(_dystack pid=231) Fitting model: CatBoost_BAG_L1 ... Training model for up to 2349.44s of the 4745.72s of remaining time.
(_dystack pid=231) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (4 workers, per: cpus=1, gpus=0, memory=2.97%)
(_ray_fit pid=1394) 	Ran out of time, early stopping on iteration 412.
(_ray_fit pid=1573) 	Ran out of time, early stopping on iteration 393. [repeated 4x across cluster]
(_dystack pid=231) 	-1.1373	 = Validation score   (-root_mean_s

In [7]:
test_data = TabularDataset(f'/kaggle/input/playground-series-s4e12/test.csv')
test_data.head()

Loaded data from: /kaggle/input/playground-series-s4e12/test.csv | Columns = 20 / 20 | Rows = 800000 -> 800000


,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type
0,1200000,28.0,Female,2310.0,NaN,4.0,Bachelor's,Self-Employed,7.657981,Rural,Basic,NaN,19.0,NaN,1.0,2023-06-04 15:21:39.245086,Poor,Yes,Weekly,House
1,1200001,31.0,Female,126031.0,Married,2.0,Master's,Self-Employed,13.381379,Suburban,Premium,NaN,14.0,372.0,8.0,2024-04-22 15:21:39.224915,Good,Yes,Rarely,Apartment
2,1200002,47.0,Female,17092.0,Divorced,0.0,PhD,Unemployed,24.354527,Urban,Comprehensive,NaN,16.0,819.0,9.0,2023-04-05 15:21:39.134960,Average,Yes,Monthly,Condo
3,1200003,28.0,Female,30424.0,Divorced,3.0,PhD,Self-Employed,5.136225,Suburban,Comprehensive,1.0,3.0,770.0,5.0,2023-10-25 15:21:39.134960,Poor,Yes,Daily,House
4,1200004,24.0,Male,10863.0,Divorced,2.0,High School,Unemployed,11.844155,Suburban,Premium,NaN,14.0,755.0,7.0,2021-11-26 15:21:39.259788,Average,No,Weekly,House


In [8]:
y_pred = predictor.predict(test_data)
y_pred.head()

0    1330.751221
1    1252.234619
2    1859.013794
3    1406.617920
4    1185.316895
Name: Premium Amount, dtype: float32

In [9]:
sample = TabularDataset(f'/kaggle/input/playground-series-s4e12/sample_submission.csv')
sample = sample.drop(columns=[label])
sample[label] = y_pred
sample.head()
sample.to_csv('submission.csv', index=False)

Loaded data from: /kaggle/input/playground-series-s4e12/sample_submission.csv | Columns = 2 / 2 | Rows = 800000 -> 800000
